In [19]:
import requests
import json
import re
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm

## 1. Request the lon and lat information of all POIs using api
### 1.1 Walk through all EU countries and request POIs of all categories in all regions in each country
The maximal number of returned POIs is 5000. Some countries might have more than 5000 POIs in the first round of request. Document those missing countries in a list

In [221]:
url_dict = {'Albania':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27AL03%27%20OR%20%22nuts_id%22%20==%20%27AL02%27%20OR%20%22nuts_id%22%20==%20%27AL01%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Austria':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27AT11%27%20OR%20%22nuts_id%22%20==%20%27AT21%27%20OR%20%22nuts_id%22%20==%20%27AT12%27%20OR%20%22nuts_id%22%20==%20%27AT31%27%20OR%20%22nuts_id%22%20==%20%27AT32%27%20OR%20%22nuts_id%22%20==%20%27AT22%27%20OR%20%22nuts_id%22%20==%20%27AT33%27%20OR%20%22nuts_id%22%20==%20%27AT34%27%20OR%20%22nuts_id%22%20==%20%27AT13%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Belgium':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27BE21%27%20OR%20%22nuts_id%22%20==%20%27BE31%27%20OR%20%22nuts_id%22%20==%20%27BE32%27%20OR%20%22nuts_id%22%20==%20%27BE22%27%20OR%20%22nuts_id%22%20==%20%27BE33%27%20OR%20%22nuts_id%22%20==%20%27BE34%27%20OR%20%22nuts_id%22%20==%20%27BE35%27%20OR%20%22nuts_id%22%20==%20%27BE23%27%20OR%20%22nuts_id%22%20==%20%27BE24%27%20OR%20%22nuts_id%22%20==%20%27BE25%27%20OR%20%22nuts_id%22%20==%20%27BE10%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Bosnia and Herzegovina':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27BA%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Bulgaria':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27BG32%27%20OR%20%22nuts_id%22%20==%20%27BG33%27%20OR%20%22nuts_id%22%20==%20%27BG31%27%20OR%20%22nuts_id%22%20==%20%27BG34%27%20OR%20%22nuts_id%22%20==%20%27BG41%27%20OR%20%22nuts_id%22%20==%20%27BG42%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Croatia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27HR05%27%20OR%20%22nuts_id%22%20==%20%27HR03%27%20OR%20%22nuts_id%22%20==%20%27HR02%27%20OR%20%22nuts_id%22%20==%20%27HR06%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Cyprus':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27CY00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Czechia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27CZ06%27%20OR%20%22nuts_id%22%20==%20%27CZ03%27%20OR%20%22nuts_id%22%20==%20%27CZ08%27%20OR%20%22nuts_id%22%20==%20%27CZ01%27%20OR%20%22nuts_id%22%20==%20%27CZ05%27%20OR%20%22nuts_id%22%20==%20%27CZ04%27%20OR%20%22nuts_id%22%20==%20%27CZ07%27%20OR%20%22nuts_id%22%20==%20%27CZ02%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Denmark':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27DK01%27%20OR%20%22nuts_id%22%20==%20%27DK04%27%20OR%20%22nuts_id%22%20==%20%27DK05%27%20OR%20%22nuts_id%22%20==%20%27DK02%27%20OR%20%22nuts_id%22%20==%20%27DK03%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Estonia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27EE00%27%20OR%20%22nuts_id%22%20==%20%27DEE0%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Finland':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27FI1C%27%20OR%20%22nuts_id%22%20==%20%27FI1B%27%20OR%20%22nuts_id%22%20==%20%27FI19%27%20OR%20%22nuts_id%22%20==%20%27FI1D%27%20OR%20%22nuts_id%22%20==%20%27FI20%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'France':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27FRF1%27%20OR%20%22nuts_id%22%20==%20%27FRI1%27%20OR%20%22nuts_id%22%20==%20%27FRK1%27%20OR%20%22nuts_id%22%20==%20%27FRD1%27%20OR%20%22nuts_id%22%20==%20%27FRC1%27%20OR%20%22nuts_id%22%20==%20%27FRH0%27%20OR%20%22nuts_id%22%20==%20%27FRB0%27%20OR%20%22nuts_id%22%20==%20%27FRF2%27%20OR%20%22nuts_id%22%20==%20%27FRM0%27%20OR%20%22nuts_id%22%20==%20%27FRC2%27%20OR%20%22nuts_id%22%20==%20%27FRY1%27%20OR%20%22nuts_id%22%20==%20%27FRY3%27%20OR%20%22nuts_id%22%20==%20%27FRD2%27%20OR%20%22nuts_id%22%20==%20%27FR10%27%20OR%20%22nuts_id%22%20==%20%27FRY4%27%20OR%20%22nuts_id%22%20==%20%27FRJ1%27%20OR%20%22nuts_id%22%20==%20%27FRI2%27%20OR%20%22nuts_id%22%20==%20%27FRF3%27%20OR%20%22nuts_id%22%20==%20%27FRY2%27%20OR%20%22nuts_id%22%20==%20%27FRY5%27%20OR%20%22nuts_id%22%20==%20%27FRJ2%27%20OR%20%22nuts_id%22%20==%20%27FRE1%27%20OR%20%22nuts_id%22%20==%20%27FRG0%27%20OR%20%22nuts_id%22%20==%20%27FRE2%27%20OR%20%22nuts_id%22%20==%20%27FRI3%27%20OR%20%22nuts_id%22%20==%20%27FRL0%27%20OR%20%22nuts_id%22%20==%20%27FRK2%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Germany':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27DEA5%27%20OR%20%22nuts_id%22%20==%20%27DE30%27%20OR%20%22nuts_id%22%20==%20%27DE40%27%20OR%20%22nuts_id%22%20==%20%27DE91%27%20OR%20%22nuts_id%22%20==%20%27DE50%27%20OR%20%22nuts_id%22%20==%20%27DED4%27%20OR%20%22nuts_id%22%20==%20%27DE71%27%20OR%20%22nuts_id%22%20==%20%27DEA4%27%20OR%20%22nuts_id%22%20==%20%27DED2%27%20OR%20%22nuts_id%22%20==%20%27DEA1%27%20OR%20%22nuts_id%22%20==%20%27DE13%27%20OR%20%22nuts_id%22%20==%20%27DE72%27%20OR%20%22nuts_id%22%20==%20%27DE60%27%20OR%20%22nuts_id%22%20==%20%27DE92%27%20OR%20%22nuts_id%22%20==%20%27DE12%27%20OR%20%22nuts_id%22%20==%20%27DE73%27%20OR%20%22nuts_id%22%20==%20%27DEB1%27%20OR%20%22nuts_id%22%20==%20%27DEA2%27%20OR%20%22nuts_id%22%20==%20%27DED5%27%20OR%20%22nuts_id%22%20==%20%27DE93%27%20OR%20%22nuts_id%22%20==%20%27DE80%27%20OR%20%22nuts_id%22%20==%20%27DE25%27%20OR%20%22nuts_id%22%20==%20%27DEA3%27%20OR%20%22nuts_id%22%20==%20%27DE22%27%20OR%20%22nuts_id%22%20==%20%27DE21%27%20OR%20%22nuts_id%22%20==%20%27DE24%27%20OR%20%22nuts_id%22%20==%20%27DE23%27%20OR%20%22nuts_id%22%20==%20%27DEB3%27%20OR%20%22nuts_id%22%20==%20%27DEC0%27%20OR%20%22nuts_id%22%20==%20%27DEE0%27%20OR%20%22nuts_id%22%20==%20%27DEF0%27%20OR%20%22nuts_id%22%20==%20%27DE27%27%20OR%20%22nuts_id%22%20==%20%27DE11%27%20OR%20%22nuts_id%22%20==%20%27DEG0%27%20OR%20%22nuts_id%22%20==%20%27DEB2%27%20OR%20%22nuts_id%22%20==%20%27DE14%27%20OR%20%22nuts_id%22%20==%20%27DE26%27%20OR%20%22nuts_id%22%20==%20%27DE94%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Greece':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27EL51%27%20OR%20%22nuts_id%22%20==%20%27EL30%27%20OR%20%22nuts_id%22%20==%20%27EL63%27%20OR%20%22nuts_id%22%20==%20%27EL53%27%20OR%20%22nuts_id%22%20==%20%27EL62%27%20OR%20%22nuts_id%22%20==%20%27EL54%27%20OR%20%22nuts_id%22%20==%20%27EL52%27%20OR%20%22nuts_id%22%20==%20%27EL43%27%20OR%20%22nuts_id%22%20==%20%27EL42%27%20OR%20%22nuts_id%22%20==%20%27EL65%27%20OR%20%22nuts_id%22%20==%20%27EL64%27%20OR%20%22nuts_id%22%20==%20%27EL61%27%20OR%20%22nuts_id%22%20==%20%27EL41%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Hungary':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27HU11%27%20OR%20%22nuts_id%22%20==%20%27HU33%27%20OR%20%22nuts_id%22%20==%20%27HU23%27%20OR%20%22nuts_id%22%20==%20%27HU21%27%20OR%20%22nuts_id%22%20==%20%27HU22%27%20OR%20%22nuts_id%22%20==%20%27HU12%27%20OR%20%22nuts_id%22%20==%20%27HU32%27%20OR%20%22nuts_id%22%20==%20%27HU31%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Ireland':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27IE06%27%20OR%20%22nuts_id%22%20==%20%27IE04%27%20OR%20%22nuts_id%22%20==%20%27IE05%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Italy':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27ITF1%27%20OR%20%22nuts_id%22%20==%20%27ITF5%27%20OR%20%22nuts_id%22%20==%20%27ITF6%27%20OR%20%22nuts_id%22%20==%20%27ITF3%27%20OR%20%22nuts_id%22%20==%20%27ITH5%27%20OR%20%22nuts_id%22%20==%20%27ITH4%27%20OR%20%22nuts_id%22%20==%20%27ITI4%27%20OR%20%22nuts_id%22%20==%20%27ITC3%27%20OR%20%22nuts_id%22%20==%20%27ITC4%27%20OR%20%22nuts_id%22%20==%20%27ITI3%27%20OR%20%22nuts_id%22%20==%20%27ITF2%27%20OR%20%22nuts_id%22%20==%20%27ITC1%27%20OR%20%22nuts_id%22%20==%20%27ITH1%27%20OR%20%22nuts_id%22%20==%20%27ITH2%27%20OR%20%22nuts_id%22%20==%20%27ITF4%27%20OR%20%22nuts_id%22%20==%20%27ITG2%27%20OR%20%22nuts_id%22%20==%20%27ITG1%27%20OR%20%22nuts_id%22%20==%20%27ITI1%27%20OR%20%22nuts_id%22%20==%20%27ITI2%27%20OR%20%22nuts_id%22%20==%20%27ITC2%27%20OR%20%22nuts_id%22%20==%20%27ITH3%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Kosovo':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27XK*%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Latvia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27LV00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Lithuania':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27LT01%27%20OR%20%22nuts_id%22%20==%20%27LT02%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Luxembourg':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27LU00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Malta':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27MT00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Montenegro':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27ME00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Netherlands':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27NL13%27%20OR%20%22nuts_id%22%20==%20%27NL23%27%20OR%20%22nuts_id%22%20==%20%27NL12%27%20OR%20%22nuts_id%22%20==%20%27NL22%27%20OR%20%22nuts_id%22%20==%20%27NL11%27%20OR%20%22nuts_id%22%20==%20%27NL42%27%20OR%20%22nuts_id%22%20==%20%27NL41%27%20OR%20%22nuts_id%22%20==%20%27NL32%27%20OR%20%22nuts_id%22%20==%20%27NL21%27%20OR%20%22nuts_id%22%20==%20%27NL31%27%20OR%20%22nuts_id%22%20==%20%27NL34%27%20OR%20%22nuts_id%22%20==%20%27NL33%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'North Macedonia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27MK00%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Norway':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27NO09%27%20OR%20%22nuts_id%22%20==%20%27NO02%27%20OR%20%22nuts_id%22%20==%20%27NO0B%27%20OR%20%22nuts_id%22%20==%20%27NO07%27%20OR%20%22nuts_id%22%20==%20%27NO08%27%20OR%20%22nuts_id%22%20==%20%27NO06%27%20OR%20%22nuts_id%22%20==%20%27NO0A%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Poland':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27PL51%27%20OR%20%22nuts_id%22%20==%20%27PL61%27%20OR%20%22nuts_id%22%20==%20%27PL81%27%20OR%20%22nuts_id%22%20==%20%27PL43%27%20OR%20%22nuts_id%22%20==%20%27PL92%27%20OR%20%22nuts_id%22%20==%20%27PL21%27%20OR%20%22nuts_id%22%20==%20%27PL52%27%20OR%20%22nuts_id%22%20==%20%27PL82%27%20OR%20%22nuts_id%22%20==%20%27PL84%27%20OR%20%22nuts_id%22%20==%20%27PL63%27%20OR%20%22nuts_id%22%20==%20%27PL62%27%20OR%20%22nuts_id%22%20==%20%27PL91%27%20OR%20%22nuts_id%22%20==%20%27PL41%27%20OR%20%22nuts_id%22%20==%20%27PL42%27%20OR%20%22nuts_id%22%20==%20%27PL71%27%20OR%20%22nuts_id%22%20==%20%27PL22%27%20OR%20%22nuts_id%22%20==%20%27PL72%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Portugal':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27PT18%27%20OR%20%22nuts_id%22%20==%20%27PT15%27%20OR%20%22nuts_id%22%20==%20%27PT16%27%20OR%20%22nuts_id%22%20==%20%27PT11%27%20OR%20%22nuts_id%22%20==%20%27PT30%27%20OR%20%22nuts_id%22%20==%20%27PT20%27%20OR%20%22nuts_id%22%20==%20%27PT17%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Romania':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27RO32%27%20OR%20%22nuts_id%22%20==%20%27RO12%27%20OR%20%22nuts_id%22%20==%20%27RO21%27%20OR%20%22nuts_id%22%20==%20%27RO11%27%20OR%20%22nuts_id%22%20==%20%27RO22%27%20OR%20%22nuts_id%22%20==%20%27RO31%27%20OR%20%22nuts_id%22%20==%20%27RO41%27%20OR%20%22nuts_id%22%20==%20%27RO42%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Serbia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27RS12%27%20OR%20%22nuts_id%22%20==%20%27RS11%27%20OR%20%22nuts_id%22%20==%20%27RS22%27%20OR%20%22nuts_id%22%20==%20%27RS21%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Slovakia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27SK01%27%20OR%20%22nuts_id%22%20==%20%27SK03%27%20OR%20%22nuts_id%22%20==%20%27SK04%27%20OR%20%22nuts_id%22%20==%20%27SK02%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Slovenia':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27SI03%27%20OR%20%22nuts_id%22%20==%20%27SI04%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Spain':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27ES61%27%20OR%20%22nuts_id%22%20==%20%27ES24%27%20OR%20%22nuts_id%22%20==%20%27ES70%27%20OR%20%22nuts_id%22%20==%20%27ES13%27%20OR%20%22nuts_id%22%20==%20%27ES41%27%20OR%20%22nuts_id%22%20==%20%27ES42%27%20OR%20%22nuts_id%22%20==%20%27ES51%27%20OR%20%22nuts_id%22%20==%20%27ES63%27%20OR%20%22nuts_id%22%20==%20%27ES64%27%20OR%20%22nuts_id%22%20==%20%27ES22%27%20OR%20%22nuts_id%22%20==%20%27ES30%27%20OR%20%22nuts_id%22%20==%20%27ES52%27%20OR%20%22nuts_id%22%20==%20%27ES43%27%20OR%20%22nuts_id%22%20==%20%27ES11%27%20OR%20%22nuts_id%22%20==%20%27ES53%27%20OR%20%22nuts_id%22%20==%20%27ES23%27%20OR%20%22nuts_id%22%20==%20%27ES21%27%20OR%20%22nuts_id%22%20==%20%27ES12%27%20OR%20%22nuts_id%22%20==%20%27ES62%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Sweden':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27SE32%27%20OR%20%22nuts_id%22%20==%20%27SE31%27%20OR%20%22nuts_id%22%20==%20%27SE21%27%20OR%20%22nuts_id%22%20==%20%27SE11%27%20OR%20%22nuts_id%22%20==%20%27SE22%27%20OR%20%22nuts_id%22%20==%20%27SE23%27%20OR%20%22nuts_id%22%20==%20%27SE12%27%20OR%20%22nuts_id%22%20==%20%27SE33%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'Switzerland':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27CH02%27%20OR%20%22nuts_id%22%20==%20%27CH03%27%20OR%20%22nuts_id%22%20==%20%27CH05%27%20OR%20%22nuts_id%22%20==%20%27CH01%27%20OR%20%22nuts_id%22%20==%20%27CH07%27%20OR%20%22nuts_id%22%20==%20%27CH06%27%20OR%20%22nuts_id%22%20==%20%27CH04%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000',
           'UK':r'https://cultural-gems.jrc.ec.europa.eu/geoserver/gems/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=gems:all_osm&srsName=EPSG:4326&CQL_FILTER=(%22category%22%20LIKE%20%27%25EUCultureFromHome%25%27%20OR%20%22category%22%20LIKE%20%27%25Aqueducts%25%27%20OR%20%22category%22%20LIKE%20%27%25Archaeological%20sites%25%27%20OR%20%22category%22%20LIKE%20%27%25Art%20gallery%25%27%20OR%20%22category%22%20LIKE%20%27%25Arts%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Attraction%25%27%20OR%20%22category%22%20LIKE%20%27%25Castles%25%27%20OR%20%22category%22%20LIKE%20%27%25Church%25%27%20OR%20%22category%22%20LIKE%20%27%25Cinema%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20gate%25%27%20OR%20%22category%22%20LIKE%20%27%25City%20walls%25%27%20OR%20%22category%22%20LIKE%20%27%25Community%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Crypt%25%27%20OR%20%22category%22%20LIKE%20%27%25Fountain%25%27%20OR%20%22category%22%20LIKE%20%27%25Historic%20building%25%27%20OR%20%22category%22%20LIKE%20%27%25Manors%25%27%20OR%20%22category%22%20LIKE%20%27%25Memorial%25%27%20OR%20%22category%22%20LIKE%20%27%25Monastery%25%27%20OR%20%22category%22%20LIKE%20%27%25Monument%25%27%20OR%20%22category%22%20LIKE%20%27%25Museum%25%27%20OR%20%22category%22%20LIKE%20%27%25Music%20venue%25%27%20OR%20%22category%22%20LIKE%20%27%25Place%20of%20worship%25%27%20OR%20%22category%22%20LIKE%20%27%25Planetarium%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20artworks%25%27%20OR%20%22category%22%20LIKE%20%27%25Public%20library%25%27%20OR%20%22category%22%20LIKE%20%27%25Social%20centre%25%27%20OR%20%22category%22%20LIKE%20%27%25Theatre%25%27%20OR%20%22category%22%20LIKE%20%27%25Viewpoint%25%27)%20AND%20(%22nuts_id%22%20==%20%27UKH2%27%20OR%20%22nuts_id%22%20==%20%27UKJ1%27%20OR%20%22nuts_id%22%20==%20%27UKD6%27%20OR%20%22nuts_id%22%20==%20%27UKK3%27%20OR%20%22nuts_id%22%20==%20%27UKD1%27%20OR%20%22nuts_id%22%20==%20%27UKF1%27%20OR%20%22nuts_id%22%20==%20%27UKK4%27%20OR%20%22nuts_id%22%20==%20%27UKK2%27%20OR%20%22nuts_id%22%20==%20%27UKH1%27%20OR%20%22nuts_id%22%20==%20%27UKL2%27%20OR%20%22nuts_id%22%20==%20%27UKE1%27%20OR%20%22nuts_id%22%20==%20%27UKM7%27%20OR%20%22nuts_id%22%20==%20%27UKH3%27%20OR%20%22nuts_id%22%20==%20%27UKK1%27%20OR%20%22nuts_id%22%20==%20%27UKD3%27%20OR%20%22nuts_id%22%20==%20%27UKJ3%27%20OR%20%22nuts_id%22%20==%20%27UKG1%27%20OR%20%22nuts_id%22%20==%20%27UKM6%27%20OR%20%22nuts_id%22%20==%20%27UKI4%27%20OR%20%22nuts_id%22%20==%20%27UKI3%27%20OR%20%22nuts_id%22%20==%20%27UKJ4%27%20OR%20%22nuts_id%22%20==%20%27UKD4%27%20OR%20%22nuts_id%22%20==%20%27UKF2%27%20OR%20%22nuts_id%22%20==%20%27UKF3%27%20OR%20%22nuts_id%22%20==%20%27UKD7%27%20OR%20%22nuts_id%22%20==%20%27UKM5%27%20OR%20%22nuts_id%22%20==%20%27UKE2%27%20OR%20%22nuts_id%22%20==%20%27UKN0%27%20OR%20%22nuts_id%22%20==%20%27UKC2%27%20OR%20%22nuts_id%22%20==%20%27UKI5%27%20OR%20%22nuts_id%22%20==%20%27UKI6%27%20OR%20%22nuts_id%22%20==%20%27UKI7%27%20OR%20%22nuts_id%22%20==%20%27UKG2%27%20OR%20%22nuts_id%22%20==%20%27UKE3%27%20OR%20%22nuts_id%22%20==%20%27UKM9%27%20OR%20%22nuts_id%22%20==%20%27UKJ2%27%20OR%20%22nuts_id%22%20==%20%27UKC1%27%20OR%20%22nuts_id%22%20==%20%27UKM8%27%20OR%20%22nuts_id%22%20==%20%27UKG3%27%20OR%20%22nuts_id%22%20==%20%27UKL1%27%20OR%20%22nuts_id%22%20==%20%27UKE4%27)&,urn:ogc:def:crs:EPSG:3857&outputFormat=application%2Fjson&maxFeatures=5000'
           }

In [222]:
def get_features_in_url(url):
    dict_features = {}
    r = requests.get(url)
    features_json = json.loads(r.text)
    
    for feature in features_json['features']:
        properties = feature['properties']
        title = properties['title']
        category = properties['category']
        source = properties['source']
        loc_lat= properties['loc_lat']
        loc_lng= properties['loc_lng']
        dict_features[title] = [title, category, source, loc_lat, loc_lng]
        
    df_features = pd.DataFrame(dict_features).T
    df_features.columns = ['title', 'category', 'source', 'lat', 'lon']

    return df_features

In [223]:
list_missing_country = []
df_features_comb = None
for country in tqdm(url_dict.keys()):
    try:
        url = url_dict[country]
        df_features_temp = get_features_in_url(url)
        df_features_comb = df_features_temp if df_features_comb is None else pd.concat([df_features_comb, df_features_temp])
    except:
        list_missing_country.append(country)
        pass
        
df_features_comb.to_csv(r'C:\1-Data\Cultural Gems\temp_0.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:34<00:00,  1.04it/s]


## 1.2 Divide the total categories into groups. Only request POIs in certain group each time to avoid the number of returned POIs exceed 5000

In [83]:
def get_category_groups(interval=3):
    # interval is the number of categories in each group
    category_list = ['Community centre', 'Social centre', 'Attraction', 'Viewpoint', 'Public library', 'Art gallery',
                 'Arts centre', 'Public artworks', 'EUCultureFromHome', 'Museum', 'Planetarium', 'Cinema', 'Music venue', 
                 'Theatre', 'Aqueducts', 'Archaeological sites', 'Castles', 'City gate', 'City walls', 'Crypt', 'Fountain', 
                 'Historic building', 'Manors', 'Memorial', 'Monument', 'Church', 'Monastery', 'Place of worship']
    num_groups = len(category_list) // interval + 1
    
    groups = []
    for i in range(num_groups):
        start = i * interval
        # Last group: take everything to the end
        if i == num_groups - 1:
            end = len(category_list)
        else:
            end = (i + 1) * interval
        group  = [i.replace(' ', '%20') for i in category_list[start:end]]
        groups.append(group)
    return groups

In [226]:
groups = get_category_groups(2)

df_missing_comb = None
for country in tqdm(list_missing_country):
    df_country_comb = None
    original_url = url_dict[country]
    for group in groups:
        category_string = '%22category%22%20LIKE%20%27%25' + '%25%27%20OR%20%22category%22%20LIKE%20%27%25'.join(group) + '%25%27'
        current_url = re.sub(r'(CQL_FILTER=\().*?(\))', rf'\1{category_string}\2', original_url)
        df_country_temp = get_features_in_url(current_url)
        df_country_comb = df_country_temp if df_country_comb is None else pd.concat([df_country_comb, df_country_temp])
    df_missing_comb = df_country_comb if df_missing_comb is None else pd.concat([df_missing_comb, df_country_comb])
    print('{} POIs were retrieved for {}'.format(len(df_country_comb), country))

df_missing_comb.to_csv(r'C:\1-Data\Cultural Gems\temp_1.csv')       

  7%|█████▉                                                                             | 1/14 [00:23<05:05, 23.48s/it]

23032 POIs were retrieved for Austria


 14%|███████████▊                                                                       | 2/14 [00:47<04:42, 23.56s/it]

19266 POIs were retrieved for Belgium


 21%|█████████████████▊                                                                 | 3/14 [01:10<04:19, 23.57s/it]

...

45214 POIs were retrieved for Spain


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [05:16<00:24, 24.68s/it]

16015 POIs were retrieved for Sweden


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [05:43<00:00, 24.54s/it]

41535 POIs were retrieved for UK


In [227]:
# Concatenante the two df of features and export
df_features = pd.concat([df_features_comb, df_missing_comb])
df_features['category_unique'] = df_features['category'].apply(lambda x: x if '|' not in str(x) else x.split('|')[0])

# merge with Doug's category
doug_category = pd.read_excel(r'C:\1-Data\Cultural Gems\cultural gems categories_drl.xlsx')
df_features = pd.merge(df_features, doug_category[['category', 'Doug’s category']], how='left', left_on='category_unique', right_on='category')
df_features.drop(columns=['category_y']).rename(columns={'category_x':'category'}).to_csv(r'C:\1-Data\Cultural Gems\all_cultural_gems_poi.csv')
print(len(df_features))

465810


## 2. Aggregate all categories by Doug’s category and by NUTS regions

In [131]:
# import and convert the dataframe of cultural gems pois to shp
df_features = pd.read_csv(r'C:\1-Data\Cultural Gems\all_cultural_gems_poi.csv', index_col=0)
gdf_features = gpd.GeoDataFrame(df_features, geometry=gpd.points_from_xy(df_features.lon, df_features.lat),
                                crs="EPSG:4326").to_crs('EPSG:3035')

In [139]:
save_folder = r'C:\2-Case studies\Cultural Gems'
nuts_folder = r'C:\1-Data\NUTS'
years = [2003, 2006, 2010, 2013, 2016, 2021, 2024]

In [ ]:
doug_categories = set(df_features['Doug’s category'])
for category in tqdm(doug_categories):
    count_cat_comb = None
    for year in years:
        nuts = gpd.read_file(nuts_folder + '\\' + 'NUTS_RG_01M_' + str(year) + '_3035.shp')
        gdf_joined = gpd.sjoin(gdf_features, nuts[['NUTS_ID', 'geometry']], how="left", predicate='within')
        gdf_joined_cat = gdf_joined[gdf_joined['Doug’s category']==category]
        count_cat = gdf_joined_cat.groupby('NUTS_ID').count()[['title']].reset_index()
        count_cat.columns = ['geo', 'ObsValue']
        count_cat['geo_source'] = 'NUTS' + str(year)
        count_cat['Indicator'] = category
        count_cat['freq'] = 'NA'
        count_cat['ObsTime'] = 2025
        count_cat['Unit'] = '-'
        count_cat_comb = count_cat if count_cat_comb is None else pd.concat([count_cat_comb, count_cat])
    count_cat_comb.reset_index().drop(columns=['index']).to_csv(save_folder + '\\' + str(category) + '.csv')